In [5]:
from itertools import product
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import pandas as pd


#权重初始化
def initialize_weights(model, init_type):
    for layer in model.modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            if init_type == 'xavier':
                nn.init.xavier_uniform_(layer.weight)
            elif init_type == 'he':
                nn.init.kaiming_uniform_(layer.weight, nonlinearity='relu')
            elif init_type == 'default':
                pass
            if layer.bias is not None:
                nn.init.constant_(layer.bias, 0)


#激活函数选择
def get_activation(name):
    if name == 'relu':
        return F.relu
    elif name == 'tanh':
        return torch.tanh
    elif name == 'sigmoid':
        return torch.sigmoid
    else:
        raise ValueError("Unsupported activation function")


#定义网络
class Net(nn.Module):
    def __init__(self, activation):
        super(Net, self).__init__()
        self.activation = activation
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.activation(x)
        x = self.conv2(x)
        x = self.activation(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


#训练
def train(model, device, train_loader, optimizer):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


#测试
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, accuracy


#主程序
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #数据加载
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=True, download=True, transform=transform),
        batch_size=64, shuffle=True) # 批量大小为64，随机打乱
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, transform=transform),
        batch_size=1000, shuffle=False)# 批量大小为1000，按顺序加载

    #参数组合
    activations = ['relu', 'tanh', 'sigmoid']
    inits = ['default', 'xavier', 'he']
    optimizers = ['sgd', 'adam', 'adadelta']
    results = []

    #遍历所有组合
    for act, init, opt in product(activations, inits, optimizers):
        print(f"\n测试组合: Activation={act}, Init={init}, Optimizer={opt}")
        model = Net(get_activation(act)).to(device)
        initialize_weights(model, init)

        #优化器选择
        optimizer = {
            'sgd': optim.SGD(model.parameters(), lr=0.01, momentum=0.9),
            'adam': optim.Adam(model.parameters(), lr=0.001),
            'adadelta': optim.Adadelta(model.parameters(), lr=1.0)
        }[opt]
        scheduler = StepLR(optimizer, step_size=1, gamma=0.7) # 学习率调度器

        #训练&测试
        for epoch in range(1, 3):  #简化为2个epoch
            train(model, device, train_loader, optimizer)
            scheduler.step()

        test_loss, accuracy = test(model, device, test_loader)
        print(f"结果: Loss={test_loss:.4f}, Accuracy={accuracy:.2f}%")

        results.append({
            'Activation': act,
            'Initialization': init,
            'Optimizer': opt,
            'Test Loss': test_loss,
            'Accuracy (%)': accuracy
        })

    #保存结果
    df = pd.DataFrame(results)
    print("\n**性能表格**")
    print(df)
    df.to_csv('mnist_performance_results.csv', index=False)
    print("结果已保存到 `mnist_performance_results.csv`")


if __name__ == '__main__':
    main()



测试组合: Activation=relu, Init=default, Optimizer=sgd
结果: Loss=0.0337, Accuracy=98.86%

测试组合: Activation=relu, Init=default, Optimizer=adam
结果: Loss=0.0313, Accuracy=98.99%

测试组合: Activation=relu, Init=default, Optimizer=adadelta
结果: Loss=0.0390, Accuracy=98.74%

测试组合: Activation=relu, Init=xavier, Optimizer=sgd
结果: Loss=0.0431, Accuracy=98.63%

测试组合: Activation=relu, Init=xavier, Optimizer=adam
结果: Loss=0.0357, Accuracy=98.85%

测试组合: Activation=relu, Init=xavier, Optimizer=adadelta
结果: Loss=0.0324, Accuracy=99.00%

测试组合: Activation=relu, Init=he, Optimizer=sgd
结果: Loss=0.0344, Accuracy=98.92%

测试组合: Activation=relu, Init=he, Optimizer=adam
结果: Loss=0.0555, Accuracy=98.37%

测试组合: Activation=relu, Init=he, Optimizer=adadelta
结果: Loss=0.0364, Accuracy=98.78%

测试组合: Activation=tanh, Init=default, Optimizer=sgd
结果: Loss=0.0479, Accuracy=98.61%

测试组合: Activation=tanh, Init=default, Optimizer=adam
结果: Loss=0.0445, Accuracy=98.54%

测试组合: Activation=tanh, Init=default, Optimizer=adadelta
结果: Los

结果分析
1. 激活函数对结果的影响
ReLU :

表现非常好，在所有优化器和初始化方法的组合中，ReLU的准确率普遍较高 (最高达到 99%)，表明其在梯度传播和网络收敛方面有优势。
最优组合是 ReLU + Xavier + Adadelta，测试损失为0.0324，准确率达到了99%。
Tanh :

其性能略逊于ReLU，准确率整体偏低（最高为98.62%）。
Tanh更适合在较浅层网络或需要对数据进行标准化的场景，但相比ReLU，在深层网络中容易出现梯度消失问题。
测试结果显示Tanh对初始化方法和优化器的选择较为敏感，表现最优的组合是 Tanh + Default + Adadelta，准确率为98.62%。
Sigmoid :

表现最差，尤其是在 Sigmoid + Xavier + Adam 组合下，准确率降到10.09%（几乎是随机分类）。这是因为Sigmoid函数在深层网络中容易出现梯度消失问题。
但在某些特定组合下（如 Sigmoid + He + Adam），测试损失低至0.0598，准确率达到了98.34%，显示在一定条件下Sigmoid仍有潜力。
2. 权重初始化对结果的影响
Default :

在ReLU和Tanh的组合中，Default初始化表现相对较好。
在Sigmoid的情况下，Default初始化的表现明显优于Xavier，测试损失更小，准确率也更高。
Xavier :

Xavier初始化在ReLU和Tanh的情况下表现较稳定，尤其是与Adadelta优化器结合时，达到最高准确率（99%）。
但在Sigmoid的情况下，Xavier初始化表现非常糟糕（如Sigmoid + Xavier + Adam的测试损失为2.302，准确率仅为10.09%），因为Sigmoid本身的梯度衰减与Xavier的初始化策略不匹配。
He :

He初始化专为ReLU设计，因此在ReLU的组合下表现非常优异，尤其是ReLU + He + SGD（准确率98.92%）。
然而在Sigmoid和Tanh的情况下，He初始化的表现不如其他方法。
3. 优化器对结果的影响
SGD :

表现稳定，但收敛速度较慢，尤其是在ReLU和Tanh的情况下，测试损失普遍较大（如Tanh + He + SGD的损失为0.0489）。
对于Sigmoid函数，SGD表现相对较差。
Adam:

通常表现优于SGD。
在ReLU的情况下，Adam优化器能达到较高的准确率（如ReLU + Default + Adam的准确率98.99%）。
但在Sigmoid + Xavier的情况下，Adam表现非常糟糕（准确率仅为10.09%），这可能是因为初始化方法和激活函数的选择导致优化陷入了局部最优解。
Adadelta:

表现非常优秀，尤其是在ReLU + Xavier的情况下达到了最高准确率99%。
Adadelta在调整学习率方面表现较好，尤其是对Sigmoid和Tanh等容易出现梯度消失问题的激活函数，具有较好的效果。
4. 结论：

ReLU 激活函数的表现最优：
它在大多数情况下能提供更好的性能，特别是与Xavier或He初始化方法结合时效果显著。
对于深度学习任务，建议优先选择ReLU。

初始化方法的选择需与激活函数匹配：
ReLU适合He和Xavier初始化，Sigmoid适合Default和He初始化，Tanh对初始化方法的选择较为宽容，但总体表现不及ReLU。

优化器选择因模型而异：
Adam优化器表现普遍较好，但对初始化方法敏感。
Adadelta在某些情况下（如ReLU + Xavier）可以实现更优的性能，尤其是对梯度消失问题具有良好效果。

Sigmoid激活函数需谨慎使用：
在深度网络中，Sigmoid激活函数表现差强人意，容易出现梯度消失问题，导致训练效果不佳。可以尝试改用ReLU或Tanh。

总体而言，最佳组合为：ReLU + Xavier + Adadelta，达到了最低测试损失（0.0324）和最高准确率（99%）。
当然这并不是普适结论，而是特定环境里的特定结果，更改参数，环境，数据集，结果可能大有不同。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import pandas as pd


#权重初始化
def initialize_weights(model, init_type):
    for layer in model.modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            if init_type == 'xavier':
                nn.init.xavier_uniform_(layer.weight)
            elif init_type == 'he':
                nn.init.kaiming_uniform_(layer.weight, nonlinearity='relu')
            elif init_type == 'default':
                pass
            if layer.bias is not None:
                nn.init.constant_(layer.bias, 0)


#激活函数选择
def get_activation(name):
    if name == 'relu':
        return F.relu
    elif name == 'tanh':
        return torch.tanh
    elif name == 'sigmoid':
        return torch.sigmoid
    else:
        raise ValueError("Unsupported activation function")


#定义网络
class Net(nn.Module):
    def __init__(self, activation):
        super(Net, self).__init__()
        self.activation = activation
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.activation(x)
        x = self.conv2(x)
        x = self.activation(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


#训练
def train(model, device, train_loader, optimizer):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


#测试
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, accuracy


#主程序
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #数据加载
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=True, download=True, transform=transform),
        batch_size=64, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, transform=transform),
        batch_size=1000, shuffle=False)

    #激活函数，初始化方式和优化器
    activation = 'sigmoid'
    init_type = 'xavier'
    optimizer_type = 'adam'

    #初始化模型
    model = Net(get_activation(activation)).to(device)
    initialize_weights(model, init_type)

    #优化器选择
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

    #训练 15 轮
    for epoch in range(1, 16):  
        print(f"Epoch {epoch}...")
        train(model, device, train_loader, optimizer)
        scheduler.step()

    #测试
    test_loss, accuracy = test(model, device, test_loader)
    print(f"Sigmoid + Xavier + Adam 组合 - 结果: Loss={test_loss:.4f}, Accuracy={accuracy:.2f}%")

    #保存结果
    results = [{
        'Activation': activation,
        'Initialization': init_type,
        'Optimizer': optimizer_type,
        'Test Loss': test_loss,
        'Accuracy (%)': accuracy
    }]
    df = pd.DataFrame(results)
    print(df)

if __name__ == '__main__':
    main()


Epoch 1...
Epoch 2...
Epoch 3...
Epoch 4...
Epoch 5...
Epoch 6...
Epoch 7...
Epoch 8...
Epoch 9...
Epoch 10...
Epoch 11...
Epoch 12...
Epoch 13...
Epoch 14...
Epoch 15...
Sigmoid + Xavier + Adam 组合 - 结果: Loss=0.0418, Accuracy=98.67%
  Activation Initialization Optimizer  Test Loss  Accuracy (%)
0    sigmoid         xavier      adam   0.041775         98.67


注意到前面Sigmoid + Xavier + Adam 组合表现非常糟糕（准确率仅为10.09%），但又不可能是sigmoid因为深度网络导致梯度消失（因为为了节省时间我只设了2个epoch），因此怀疑是性能差的主要原因可能是训练时间不足，模型还没有充分学习和调整。而结果也正如所料，提高了训练轮数后，该组合表现也还不错。